# 电影分类中的关联规则和频繁项集

In [1]:
# 分析MovieLens 电影分类中的频繁项集和关联规则
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 数据加载
movies = pd.read_csv('C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Avalon Ding\\Kaikeba\\L3\\movies.csv')
#print(movies.head())
# 将genres进行one-hot编码（离散特征有多少取值，就用多少维来表示这个特征）
movies_hot_encoded = movies.drop('genres',1).join(movies.genres.str.get_dummies(sep='|'))
pd.options.display.max_columns=100
print(movies_hot_encoded.head())

# 将movieId, title设置为index
movies_hot_encoded.set_index(['movieId','title'],inplace=True)
#print(movies_hot_encoded.head())
# 挖掘频繁项集，最小支持度为0.02
itemsets = apriori(movies_hot_encoded,use_colnames=True, min_support=0.02)
# 按照支持度从大到小进行时候粗
itemsets = itemsets.sort_values(by="support" , ascending=False) 
print('-'*20, '频繁项集', '-'*20)
print(itemsets)
# 根据频繁项集计算关联规则，设置最小提升度为2
rules =  association_rules(itemsets, metric='lift', min_threshold=2)
# 按照提升度从大到小进行排序
rules = rules.sort_values(by="lift" , ascending=False) 
#rules.to_csv('./rules.csv')
print('-'*20, '关联规则', '-'*20)
print(rules)

   movieId                               title  (no genres listed)  Action  \
0        1                    Toy Story (1995)                   0       0   
1        2                      Jumanji (1995)                   0       0   
2        3             Grumpier Old Men (1995)                   0       0   
3        4            Waiting to Exhale (1995)                   0       0   
4        5  Father of the Bride Part II (1995)                   0       0   

   Adventure  Animation  Children  Comedy  Crime  Documentary  Drama  Fantasy  \
0          1          1         1       1      0            0      0        1   
1          1          0         1       0      0            0      0        1   
2          0          0         0       1      0            0      0        0   
3          0          0         0       1      0            0      1        0   
4          0          0         0       1      0            0      0        0   

   Film-Noir  Horror  IMAX  Musical  Mystery

# 爬虫网页扒取豆瓣

In [4]:
# -*- coding: utf-8 -*-
# 下载某个演员/导演的电影数据集
from lxml import etree
import time
from selenium import webdriver
import pandas as pd

"""
这里我们需要使用ChromeDrvier来做模拟
Step1，打开谷歌浏览器， 在地址栏输入 chrome://version/  查看版本信息
Step2，ChromeDriver版本下载地址：http://chromedriver.storage.googleapis.com/index.html
Step3，放到Python\Lib\site-packages相应路径
"""
chrome_driver = r"./chromedriver.exe"
driver = webdriver.Chrome(executable_path=chrome_driver)
# 设置想要下载的导演 数据集
director = u'徐峥'
base_url = 'https://movie.douban.com/subject_search?search_text='+director+'&cat=1002&start='

movie_actors = {}
# 下载指定页面的数据
def download(request_url):
    driver.get(request_url)
    time.sleep(1)
    html = driver.find_element_by_xpath("//*").get_attribute("outerHTML")
    html = etree.HTML(html)
    # 设置电影名称，导演演员 的XPATH
    movie_lists = html.xpath("/html/body/div[@id='wrapper']/div[@id='root']/div[1]//div[@class='item-root']/div[@class='detail']/div[@class='title']/a[@class='title-text']")
    name_lists = html.xpath("/html/body/div[@id='wrapper']/div[@id='root']/div[1]//div[@class='item-root']/div[@class='detail']/div[@class='meta abstract_2']")
    # 获取返回的数据个数
    num = len(movie_lists)
    
    if num > 15: #第一页会有16条数据
        # 默认第一个不是，所以需要去掉
        movie_lists = movie_lists[1:]
        name_lists = name_lists[1:]
    for (movie, name_list) in zip(movie_lists, name_lists):
        # 会存在数据为空的情况
        if name_list.text is None: 
            continue
        # 显示下演员名称
        names = name_list.text.split('/')
        movie_actors[movie.text] = name_list.text.replace(" ", "")
        print(name_list.text.replace(" ", ""))
    print('OK') # 代表这页数据下载成功
    if num >= 15:
        # 继续下一页
        return True
    else:
        # 没有下一页
        return False

# 开始的ID为0，每页增加15
start = 0
while start<10000: #最多抽取1万部电影
    request_url = base_url + str(start)
    # 下载数据，并返回是否有下一页
    flag = download(request_url)
    if flag:
        start = start + 15
    else:
        break

# 将字典类型转化为DataFrame
movie_actors = pd.DataFrame(movie_actors, index=[0])
#print(movie_actors)
# DataFrame 行列转换
movie_actors = pd.DataFrame(movie_actors.values.T, index=movie_actors.columns, columns=movie_actors.index)
movie_actors.index.name = 'title'
movie_actors.set_axis(['actors'], axis='columns', inplace=True)
movie_actors.to_csv('./movie_actors.csv')
print('finished')

陈凯歌/张一白/管虎/薛晓路/徐峥/宁浩/文牧野/黄渤/张译/韩昊霖/杜江/葛优/刘昊然/宋佳/王千源
文牧野/徐峥/王传君/周一围/谭卓/章宇/杨新鸣/王佳佳/王砚辉
宁浩/黄渤/沈腾/汤姆·派福瑞/马修·莫里森/徐峥/于和伟/雷佳音/刘桦
黄渤/舒淇/王宝强/张艺兴/于和伟/王迅/李勤勤/李又麟
贾樟柯/赵涛/廖凡/徐峥/梁嘉艳/刁亦男/张一白/丁嘉丽/张译
宁浩/郭涛/刘桦/连晋/黄渤/徐峥/优恵/罗兰/王迅
宁浩/黄渤/徐峥/袁泉/周冬雨/陶慧/岳小军/沈腾/张俪
徐峥/袁泉/沈腾/吴云芳/陈奇
宁浩/徐峥/黄渤/余男/多布杰/王双宝/巴多/杨新鸣/郭虹
徐峥/王宝强/黄渤/陶虹/谢楠/范冰冰
徐峥/赵薇/包贝尔/杜鹃/葛民辉/李灿森/潘虹/赵有亮
宁浩/黄渤/戎祥/九孔/徐峥/王双宝/巴多/董立范/高捷
苏伦/雷佳音/佟丽娅/张衣/于和伟/王正佳/陶虹/李念/李光洁
叶伟民/徐峥/王宝强/李曼/李小璐/左小青/张歆艺/黄小蕾/马健
陈正道/徐峥/莫文蔚/胡静/吕中/王耀庆/杨凯迪
OK
彭浩翔/杨千嬅/余文乐/杨幂/徐峥/陈逸宁/林兆霞/谷德昭/詹瑞文
任鹏远/徐峥/王丽坤/王砚辉/段博文/任达华/于和伟/朱珠/赵达
谭晓虹/于谦/李诞/吴昕/池子/马东/王建国/程璐/庞博
韩三平/黄建新/刘烨/冯远征/张嘉译/陈坤/马少骅/李沁/周润发/赵本山
张建亚/徐峥/刘仪伟/宁静/白冰/伊能静/车永莉/沈星/宋佳
杨庆/徐峥/李小璐/乔任梁/杨青/张嘉译/赵英俊
邵晓黎/徐峥/张子枫/王宣予/张颂文/王太利/肖央/岳小军/杜家毅
张建亚/林嘉欣/邓超/黄渤/范伟/古巨基/黄磊/苏有朋/佟大为
范小天/叶崇铭/梦继/徐峥/陶虹/孙兴/李立群/陈红/翁虹/屈中恒/钮承泽
嵇天毅/于和伟/张丹峰/侯梦莎/傅浤鸣/徐洪浩/吴秀波/徐峥/侯勇
金依萌/范冰冰/李治廷/吴佩慈/蒋劲夫/丹尼尔·海尼/刘维/倪虹洁/黎明
霍建起/黄晓明/徐峥/蒲巴甲/罗晋/汤镇业/锦荣/谭凯/姜超
吴彤/章子怡/徐峥/吴秀波/张国立/伊一/刘天池/陈凯歌/许鞍华
孙皓/徐峥/宋佳/梅婷/张歆艺/马苏/车晓/隋兰/王茂蕾
孙太泉/杨军/张庭/徐峥/万弘杰/刘莉莉/孙宝光/史可/李倩/杨俊毅
OK
任静/严敏/黄渤/孙红雷/黄磊/罗志祥/张艺兴/王迅/陈乔恩/郭涛
李海蜀/徐峥/许绍雄/林鹏/李凤